In [1]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary 

# b0-13, t, a0-4

# Design 1: (3-4-3) 
# 4 3-bit in place control gates (b0-11)->(a0-3)
# 1 4-bit in place control gate taking 2 remaining bits and 2 auxiliary bits (b12-13, a0-1) -> a4
# 3 3-bit in place control gate that takes (a2-4) -> t

#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram 


In [2]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')

In [3]:
# no optimization - design 0
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
qc.mcx(control_qubits, target_qubit[0], ancilla_qubits=ancilla_qubits, mode='recursion')
qc.measure(target_qubit[0], classical_bits[0])

In [4]:
qc.draw()

c_0: ──■──────
         │      
  c_1: ──■──────
         │      
  c_2: ──■──────
         │      
  c_3: ──■──────
         │      
  c_4: ──■──────
         │      
  c_5: ──■──────
         │      
  c_6: ──■──────
         │      
  c_7: ──■──────
         │      
  c_8: ──■──────
         │      
  c_9: ──■──────
         │      
 c_10: ──■──────
         │      
 c_11: ──■──────
         │      
 c_12: ──■──────
         │      
 c_13: ──■──────
       ┌─┴──┐┌─┐
    t: ┤0   ├┤M├
       │  X │└╥┘
  a_0: ┤1   ├─╫─
       └────┘ ║ 
  a_1: ───────╫─
              ║ 
  a_2: ───────╫─
              ║ 
  a_3: ───────╫─
              ║ 
  a_4: ───────╫─
              ║ 
out: 1/═══════╩═
              0

In [5]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

890
OrderedDict([('u', 712), ('cx', 656), ('measure', 1)])


In [7]:
# design 1 
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(14):
    qc.x(control_qubits[i])

for i in range(4):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])

qc.mcx(control_qubits[12:14] + ancilla_qubits[0:2], ancilla_qubits[4])
qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])
qc.draw()

┌───┐                                 
  c_0: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_1: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_2: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_3: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_4: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_5: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_6: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_7: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_8: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_9: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_10: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_11: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_12: ┤ X ├──┼────┼────┼────┼────■──────────
       ├───┤  │    │    │    │    │          
 c_13: ┤ X ├──┼────┼────┼────┼────■──────────
       └───┘  │    │    │    │    │  ┌───┐┌─┐
    t: ───────┼────┼────┼────┼────┼──┤ X ├┤M├
            ┌─┴─┐  │    │    │    │  └─┬─┘└╥┘
  a_0: ─────┤ X ├──┼────┼────┼────■────┼───╫─
            └───┘┌─┴─┐  │    │    │    │   ║ 
  a_1: ──────────┤ X ├──┼────┼────■────┼───╫─
                 └───┘┌─┴─┐  │    │    │   ║ 
  a_2: ───────────────┤ X ├──┼────┼────■───╫─
                      └───┘┌─┴─┐  │    │   ║ 
  a_3: ────────────────────┤ X ├──┼────■───╫─
                           └───┘┌─┴─┐  │   ║ 
  a_4: ─────────────────────────┤ X ├──■───╫─
                                └───┘      ║ 
out: 1/════════════════════════════════════╩═
                                           0

In [8]:
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

115
OrderedDict([('u', 119), ('cx', 106), ('measure', 1)])


In [10]:
# design 2 - currently best design
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(14):
    qc.x(control_qubits[i])

for i in range(3):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])
# 0 - 8
qc.mcx(control_qubits[9:13], ancilla_qubits[3])

qc.mcx(ancilla_qubits[0:2] + control_qubits[13:14], ancilla_qubits[4])

qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])

# uncompute
qc.mcx(ancilla_qubits[0:2] + control_qubits[13:14], ancilla_qubits[4])
for i in range(3):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])
# 0 - 8
qc.mcx(control_qubits[9:13], ancilla_qubits[3])

qc.draw()

┌───┐                                                          
  c_0: ┤ X ├──■───────────────────────────────────────────────■───────
       ├───┤  │                                               │       
  c_1: ┤ X ├──■───────────────────────────────────────────────■───────
       ├───┤  │                                               │       
  c_2: ┤ X ├──■───────────────────────────────────────────────■───────
       ├───┤  │                                               │       
  c_3: ┤ X ├──┼────■──────────────────────────────────────────┼────■──
       ├───┤  │    │                                          │    │  
  c_4: ┤ X ├──┼────■──────────────────────────────────────────┼────■──
       ├───┤  │    │                                          │    │  
  c_5: ┤ X ├──┼────■──────────────────────────────────────────┼────■──
       ├───┤  │    │                                          │    │  
  c_6: ┤ X ├──┼────┼────■───────────────────────────■─────────┼────┼──
       ├───┤  │    │    │                           │         │    │  
  c_7: ┤ X ├──┼────┼────■───────────────────────────■─────────┼────┼──
       ├───┤  │    │    │                           │         │    │  
  c_8: ┤ X ├──┼────┼────■───────────────────────────■─────────┼────┼──
       ├───┤  │    │    │                           │         │    │  
  c_9: ┤ X ├──┼────┼────┼────■──────────────────────┼────■────┼────┼──
       ├───┤  │    │    │    │                      │    │    │    │  
 c_10: ┤ X ├──┼────┼────┼────■──────────────────────┼────■────┼────┼──
       ├───┤  │    │    │    │                      │    │    │    │  
 c_11: ┤ X ├──┼────┼────┼────■──────────────────────┼────■────┼────┼──
       ├───┤  │    │    │    │                      │    │    │    │  
 c_12: ┤ X ├──┼────┼────┼────■──────────────────────┼────■────┼────┼──
       ├───┤  │    │    │    │                      │    │    │    │  
 c_13: ┤ X ├──┼────┼────┼────┼────■────────────■────┼────┼────┼────┼──
       └───┘  │    │    │    │    │  ┌───┐┌─┐  │    │    │    │    │  
    t: ───────┼────┼────┼────┼────┼──┤ X ├┤M├──┼────┼────┼────┼────┼──
            ┌─┴─┐  │    │    │    │  └─┬─┘└╥┘  │    │    │  ┌─┴─┐  │  
  a_0: ─────┤ X ├──┼────┼────┼────■────┼───╫───■────┼────┼──┤ X ├──┼──
            └───┘┌─┴─┐  │    │    │    │   ║   │    │    │  └───┘┌─┴─┐
  a_1: ──────────┤ X ├──┼────┼────■────┼───╫───■────┼────┼───────┤ X ├
                 └───┘┌─┴─┐  │    │    │   ║   │  ┌─┴─┐  │       └───┘
  a_2: ───────────────┤ X ├──┼────┼────■───╫───┼──┤ X ├──┼────────────
                      └───┘┌─┴─┐  │    │   ║   │  └───┘┌─┴─┐          
  a_3: ────────────────────┤ X ├──┼────■───╫───┼───────┤ X ├──────────
                           └───┘┌─┴─┐  │   ║ ┌─┴─┐     └───┘          
  a_4: ─────────────────────────┤ X ├──■───╫─┤ X ├────────────────────
                                └───┘      ║ └───┘                    
out: 1/════════════════════════════════════╩══════════════════════════
                                           0

In [11]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

151
OrderedDict([('u', 223), ('cx', 198), ('measure', 1)])


In [12]:
# design 3
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(14):
    qc.x(control_qubits[i])

for i in range(7):
    qc.mcx(control_qubits[2*i:2*(i+1)], ancilla_qubits[i], mode='recursion')
for i in range(8):
    qc.mcx(control_qubits[2*i:2*(i+1)], ancilla_qubits[i], mode='recursion')
# 0 - 11
qc.mcx(control_qubits[12:14] + [ancilla_qubits[0]], ancilla_qubits[4], mode='recursion')
qc.mcx(ancilla_qubits[1:5], target_qubit[0], mode='recursion')
qc.measure(target_qubit[0], classical_bits[0])
qc.draw()

IndexError: list index out of range

In [39]:
# test efficiency of various sizes with no-ancilla
numControls = 10
modes = ['noancilla', 'recursion', 'v-chain', 'v-chain-dirty']
for m in modes:
    print(f"Mode: {m}")
    for i in range(1, numControls):
        ancilla_qubits = QuantumRegister(15, name='a')
        control_qubits = QuantumRegister(i, name="c")
        qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
        for j in range(i):
            qc.x(control_qubits[j])
        qc.mcx(control_qubits, ancilla_qubits[0], ancilla_qubits=ancilla_qubits[1:], mode=m)
        qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
        if (m == 'noancilla'):
            num_ancilla = 0
        elif (m == 'recursion'):
            num_ancilla = 0 if i < 5 else 1
        elif (m == 'v-chain' or m =='v-chain-dirty'):
            num_ancilla = i - 2 if i > 2 else 0
        print(f"Controls: {i}, Depth: {qc_transpiled.depth()}, Ancilla: {num_ancilla}, Ops: {qc_transpiled.count_ops()}")
    print()

Mode: noancilla
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65, Ancilla: 0, Ops: OrderedDict([('u', 44), ('cx', 36)])
Controls: 5, Depth: 155, Ancilla: 0, Ops: OrderedDict([('u', 98), ('cx', 92)])
Controls: 6, Depth: 315, Ancilla: 0, Ops: OrderedDict([('u', 195), ('cx', 188)])
Controls: 7, Depth: 635, Ancilla: 0, Ops: OrderedDict([('u', 388), ('cx', 380)])
Controls: 8, Depth: 1275, Ancilla: 0, Ops: OrderedDict([('u', 773), ('cx', 764)])
Controls: 9, Depth: 2555, Ancilla: 0, Ops: OrderedDict([('u', 1542), ('cx', 1532)])

Mode: recursion
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65